In [1]:
import csv
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Image
from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp

start = time.time()

In [2]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
           "01_NDVI_TS/00_TS_Batches_70_cloud/"

output_dir = data_dir
plot_dir_base = data_dir + "plots/"

In [3]:
file_names = ["batch_A_2017_TS.csv"]
file_N = file_names[0]
an_EE_TS = pd.read_csv(data_dir + file_N)

####################################################################################
###
###                   process data
###
####################################################################################
#
#  Pick Grant
#
an_EE_TS = an_EE_TS[an_EE_TS['county']=="Grant"]
print (an_EE_TS.county.unique())

#
# The following columns do not exist in the old data
#
an_EE_TS['CovrCrp'] = "NA" 
an_EE_TS['DataSrc'] = "NA" 

an_EE_TS_NDVI = rc.initial_clean_NDVI(an_EE_TS)
an_EE_TS_EVI = rc.initial_clean_EVI(an_EE_TS)
an_EE_TS_NDVI.head(2)

['Grant']


,B1,B10,B11,B12,B2,B3,B4,B5,B6,B7,...,Shp_Lng,Source,TRS,TtlAcrs,county,doy,year,geo,CovrCrp,DataSrc
0,0.613043,0.002464,0.118556,0.111530,0.599107,0.530315,0.607342,0.634768,0.649198,0.663978,...,1930.161276,WSDA,T16R23E28,48,Grant,2.0,2017.0,"{""type"":""Polygon"",""coordinates"":[[[-119.945310...",NA,NA
3,0.176401,0.009207,0.217102,0.143611,0.139706,0.118812,0.103657,0.144097,0.228967,0.251434,...,1930.161276,WSDA,T16R23E28,48,Grant,59.0,2017.0,"{""type"":""Polygon"",""coordinates"":[[[-119.945310...",NA,NA


In [14]:
### List of unique polygons
polygon_list = an_EE_TS_NDVI['geo'].unique()
print(len(polygon_list))

6576


In [17]:
counter = 0
for a_poly in polygon_list:
    if (counter%1000 == 0):
        print (counter)
    counter += 1
    curr_field_NDVI = an_EE_TS_NDVI[an_EE_TS_NDVI['geo']==a_poly]
    curr_field_EVI = an_EE_TS_EVI[an_EE_TS_EVI['geo']==a_poly]

    year = int(curr_field_NDVI['year'].unique())
    plant = curr_field_NDVI['CropTyp'].unique()[0]
    
    # Take care of names, replace "/" and "," and " " by "_"
    plant = plant.replace("/", "_")
    plant = plant.replace(",", "_")
    plant = plant.replace(" ", "_")

    county = curr_field_NDVI['county'].unique()[0]
    TRS = curr_field_NDVI['TRS'].unique()[0]

    x_NDVI = curr_field_NDVI['doy']
    y_NDVI = curr_field_NDVI['NDVI']

    x_EVI = curr_field_EVI['doy']
    y_EVI = curr_field_EVI['EVI']

    #############################################
    ###
    ###             plot
    ###
    #############################################        
    sub_out = "/plant_based_plots/" + plant + "/"
    plot_path = plot_dir_base + sub_out
    os.makedirs(plot_path, exist_ok=True)
    if (len(os.listdir(plot_path))<100):
        plot_title = county + ", " + plant + ", " + str(year) + " (" + TRS + ")"
        sb.set();
        fig, ax = plt.subplots(figsize=(8,6));
        ax.plot(x_NDVI, y_NDVI, label="NDVI");
        ax.plot(x_EVI, y_EVI, 'r', label="EVI")
        ax.set_title(plot_title);
        ax.set(xlabel='DoY', ylabel='NDVI & EVI')
        ax.legend(loc="best");

        fig_name = plot_path + county + "_" + plant + "_" + str(year) + "_" + str(counter) + '.png'
        plt.savefig(fname = fig_name, \
                    dpi=400,
                    bbox_inches='tight')
        plt.close()
        del(plot_path, sub_out, county, plant, year)

####################################################################################
###
###                   Compute double crop area
###
####################################################################################


end_time = time.time()
print(end_time - start_time)


0


NameError: name 'WSDA_df' is not defined